In [46]:
import seaborn as sns
import pandas as pd
import re
import matplotlib as mpl
from matplotlib import rc
import missingno as msno
import warnings
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
warnings.filterwarnings('ignore')

# matplotlib 한글 사용을 위한 코드
rc('font', family='AppleGothic')

In [47]:
final_df = pd.read_csv('../../final_df_13/final_df_13.csv').drop(columns='Unnamed: 0')
X_train = pd.read_csv('../../final_df_13/X_train_13.csv').drop(columns='Unnamed: 0')
y_train = pd.read_csv('../../final_df_13/y_train_13.csv').drop(columns='Unnamed: 0')
X_test = pd.read_csv('../../final_df_13/X_test_13.csv').drop(columns='Unnamed: 0')
y_test = pd.read_csv('../../final_df_13/y_test_13.csv').drop(columns='Unnamed: 0')

In [48]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925 entries, 0 to 924
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 925 non-null    object 
 1   director              925 non-null    object 
 2   distributor           925 non-null    object 
 3   distributor_group     925 non-null    object 
 4   release_date          925 non-null    int64  
 5   release_season        925 non-null    object 
 6   release               925 non-null    object 
 7   form                  925 non-null    object 
 8   nation                925 non-null    object 
 9   screen                925 non-null    int64  
 10  rev_nationwide        925 non-null    float64
 11  aud_nationwide        925 non-null    int64  
 12  rev_seoul             925 non-null    int64  
 13  rev_seoul.1           925 non-null    int64  
 14  genre                 925 non-null    object 
 15  rate                  9

In [50]:
len(X_train), len(X_test), len(y_train), len(y_test),

(740, 185, 740, 185)

In [51]:
df = pd.concat([X_train, y_train], axis=1)

In [52]:
df.head(1)

,title,director,distributor,distributor_group,release_date,release_season,release,form,nation,screen,...,actor_power,director_count,director_movie_count,actor_movie_count,before_news,after_news,holiday,dist_cat,aud_nationwide,log_aud
0,인사이드 아웃,피트 닥터,월트디즈니컴퍼니코리아(주),B,20150709,summer,개봉영화,장편,미국,777,...,0,1,0,0,67.0,352.0,0,B,4968687,15.418666


In [58]:
df = df.drop(columns=['distributor_group', 'release', 'form', 'rev_nationwide', 'rev_seoul', 'rev_seoul.1', ''])

Index(['title', 'director', 'distributor', 'distributor_group', 'release_date',
       'release_season', 'release', 'form', 'nation', 'screen',
       'rev_nationwide', 'rev_seoul', 'rev_seoul.1', 'genre', 'rate', 'type',
       'expert_rating', 'starring_actors', 'running_times', 'audience_rating',
       'rating_audiences', 'realease_year', 'director_power', 'sequel',
       'original', 'actor_power', 'director_count', 'director_movie_count',
       'actor_movie_count', 'before_news', 'after_news', 'holiday', 'dist_cat',
       'aud_nationwide', 'log_aud'],
      dtype='object')

In [59]:
df['type']

0         일반영화
1      독립/예술영화
2         일반영화
3      독립/예술영화
4         일반영화
        ...   
735       일반영화
736       일반영화
737       일반영화
738       일반영화
739    독립/예술영화
Name: type, Length: 740, dtype: object